In [2]:
class PresentCipher:
    # S-box
    SBOX = [
        0xC, 0x5, 0x6, 0xB,
        0x9, 0x0, 0xA, 0xD,
        0x3, 0xE, 0xF, 0x8,
        0x4, 0x7, 0x1, 0x2
    ]
    # Inverse S-box (computed after SBOX is defined)
    SBOX_INV = [0]*16
    for i, val in enumerate(SBOX):
        SBOX_INV[val] = i

    # P-layer and inverse P-layer
    PBOX = [
         0, 16, 32, 48,  1, 17, 33, 49,
         2, 18, 34, 50,  3, 19, 35, 51,
         4, 20, 36, 52,  5, 21, 37, 53,
         6, 22, 38, 54,  7, 23, 39, 55,
         8, 24, 40, 56,  9, 25, 41, 57,
        10, 26, 42, 58, 11, 27, 43, 59,
        12, 28, 44, 60, 13, 29, 45, 61,
        14, 30, 46, 62, 15, 31, 47, 63
    ]
    PBOX_INV = [0]*64
    for i, p in enumerate(PBOX):
        PBOX_INV[p] = i

    def __init__(self, key: int):
        if key.bit_length() > 80:
            raise ValueError("Key must be 80 bits or less")
        self.key = key
        self.round_keys = self._generate_round_keys(key)

    def _rotate_key(self, key: int) -> int:
        # Rotate left by 61 bits for 80-bit key
        left = (key << 61) & ((1 << 80) - 1)
        right = key >> (80 - 61)
        return left | right

    def _generate_round_keys(self, key: int):
        keys = []
        for round_counter in range(1, 32):
            round_key = key >> 16
            keys.append(round_key & ((1 << 64) - 1))

            key = self._rotate_key(key)

            # Apply S-box to top 4 bits
            top_nibble = (key >> 76) & 0xF
            top_nibble = self.SBOX[top_nibble]
            key &= ~(0xF << 76)
            key |= top_nibble << 76

            # XOR round counter with bits 19..15
            rc = round_counter & 0x1F
            key ^= rc << 15
        return keys

    def _sbox_layer(self, state: int) -> int:
        output = 0
        for i in range(16):
            nibble = (state >> (i*4)) & 0xF
            output |= self.SBOX[nibble] << (i*4)
        return output

    def _sbox_inv_layer(self, state: int) -> int:
        output = 0
        for i in range(16):
            nibble = (state >> (i*4)) & 0xF
            output |= self.SBOX_INV[nibble] << (i*4)
        return output

    def _pbox_layer(self, state: int) -> int:
        output = 0
        for i in range(64):
            bit = (state >> i) & 0x1
            output |= bit << self.PBOX[i]
        return output

    def _pbox_inv_layer(self, state: int) -> int:
        output = 0
        for i in range(64):
            bit = (state >> i) & 0x1
            output |= bit << self.PBOX_INV[i]
        return output

    def encrypt(self, plaintext: int) -> int:
        if plaintext.bit_length() > 64:
            raise ValueError("Plaintext must be 64 bits or less")

        state = plaintext
        for i in range(31):
            state ^= self.round_keys[i]
            state = self._sbox_layer(state)
            state = self._pbox_layer(state)
        state ^= self.round_keys[-1]
        return state

    def decrypt(self, ciphertext: int) -> int:
        if ciphertext.bit_length() > 64:
            raise ValueError("Ciphertext must be 64 bits or less")

        state = ciphertext
        state ^= self.round_keys[-1]
        for i in reversed(range(31)):
            state = self._pbox_inv_layer(state)
            state = self._sbox_inv_layer(state)
            state ^= self.round_keys[i]
        return state

    @staticmethod
    def int_to_bitstring(value: int, bits: int) -> str:
        return format(value, f'0{bits}b')

    @staticmethod
    def bitstring_to_int(bitstr: str) -> int:
        return int(bitstr, 2)


if __name__ == "__main__":
    plaintext = 0x0000000000000000
    key = 0x00000000000000000000  # 80-bit zero key

    cipher = PresentCipher(key)

    ciphertext = cipher.encrypt(plaintext)
    print("Plaintext:  0x{0:016X}".format(plaintext))
    print("Key:        0x{0:020X}".format(key))
    print("Ciphertext: 0x{0:016X}".format(ciphertext))

    decrypted = cipher.decrypt(ciphertext)
    print("Decrypted:  0x{0:016X}".format(decrypted))
    assert decrypted == plaintext, "Decryption failed!"

    print("Plaintext bits:  ", PresentCipher.int_to_bitstring(plaintext, 64))
    print("Ciphertext bits: ", PresentCipher.int_to_bitstring(ciphertext, 64))



Plaintext:  0x0000000000000000
Key:        0x00000000000000000000
Ciphertext: 0xB3708A428C1B698C
Decrypted:  0x0000000000000000
Plaintext bits:   0000000000000000000000000000000000000000000000000000000000000000
Ciphertext bits:  1011001101110000100010100100001010001100000110110110100110001100


In [10]:
import secrets

plaintext_bytes = b"A un amigo perdido"
plaintext_int = int.from_bytes(plaintext_bytes[:8], byteorder='big')

# Generate random 80-bit key
random_key = secrets.randbits(80)

cipher = PresentCipher(random_key)
ciphertext = cipher.encrypt(plaintext_int)
decrypted_int = cipher.decrypt(ciphertext)

# Convert decrypted int back to bytes
decrypted_bytes = decrypted_int.to_bytes(8, byteorder='big')

print(f"Random key: 0x{random_key:020X}")
print("Plaintext bits:  ", PresentCipher.int_to_bitstring(plaintext_int, 64))
print("Ciphertext bits: ", PresentCipher.int_to_bitstring(ciphertext, 64))
print("Decrypted bits:  ", PresentCipher.int_to_bitstring(decrypted_int, 64))
print("Decrypted text:  ", decrypted_bytes.decode(errors='ignore'))  # decode safely


Random key: 0x800DD04AB1F0ACF3379E
Plaintext bits:   0100000100100000011101010110111000100000011000010110110101101001
Ciphertext bits:  1101110101101110010001110110000000100011000011110110000000010111
Decrypted bits:   0100000100100000011101010110111000100000011000010110110101101001
Decrypted text:   A un ami


In [11]:
def split_blocks(data: bytes, block_size: int = 8):
    return [data[i:i+block_size] for i in range(0, len(data), block_size)]

plaintext_bytes = b"A un amigo perdido"
blocks = split_blocks(plaintext_bytes)

random_key = secrets.randbits(80)
cipher = PresentCipher(random_key)

ciphertext_blocks = []
decrypted_blocks = []

for block in blocks:
    # Pad block with zeros if less than 8 bytes
    block_padded = block.ljust(8, b'\x00')
    block_int = int.from_bytes(block_padded, byteorder='big')

    ciphertext = cipher.encrypt(block_int)
    ciphertext_blocks.append(ciphertext)

    decrypted_int = cipher.decrypt(ciphertext)
    decrypted_block = decrypted_int.to_bytes(8, byteorder='big')
    decrypted_blocks.append(decrypted_block)

# Join decrypted blocks and remove possible padding zeros at the end
decrypted_message = b''.join(decrypted_blocks).rstrip(b'\x00')

print(f"Random key: 0x{random_key:020X}")
print("Plaintext: ", plaintext_bytes)
print("Decrypted:", decrypted_message)
print("Decrypted text:", decrypted_message.decode(errors='ignore'))

Random key: 0xCD57350C6C4126B06095
Plaintext:  b'A un amigo perdido'
Decrypted: b'A un amigo perdido'
Decrypted text: A un amigo perdido
